# Batch Whitening Layer
The purpose of this notebook is to implement the batch whitening layer.   
The implementation is inspired by the implementation of BatchNorm layer from [this reference](https://d2l.ai/chapter_convolutional-modern/batch-norm.html)


In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import numpy as np 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print("Device to use:", device)

%load_ext autoreload
%autoreload 2

## Batch Normalization  
Lets start with implementing BatchNorm from scratch and test it on a simple dataset.


### Option 1

In [ ]:
def batch_norm(X, gamma, beta, running_mean, running_var, eps, momentum):
    # Use is_grad_enabled to determine whether we are in training mode
    assert len(X.shape) in (2, 4)
    if len(X.shape) == 2:
        shape = (1, X.shape[1])
    else:
        shape = (1, X.shape[1], 1, 1)

    if not torch.is_grad_enabled():
        print('grad disabled')
        # In prediction mode, use mean and variance obtained by moving average
        X_hat = (X - running_mean) / torch.sqrt(running_var + eps)
    else:
        print('grad enabled')
        if len(X.shape) == 2:
            # When using a fully connected layer, calculate the mean and
            # variance on the feature dimension
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # When using a two-dimensional convolutional layer, calculate the
            # mean and variance on the channel dimension (axis=1). Here we
            # need to maintain the shape of X, so that the broadcasting
            # operation can be carried out later
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # In training mode, the current mean and variance are used
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # Update the mean and variance using moving average
        running_mean = (1.0 - momentum) * running_mean + momentum * mean
        running_var = (1.0 - momentum) * running_var + momentum * var
    Y = gamma.view(shape) * X_hat + beta.view(shape)  # Scale and shift
    return Y, running_mean.data, running_var.data


class BatchNorm(nn.Module):
    # num_features: the number of outputs for a fully connected layer or the
    # number of output channels for a convolutional layer. num_dims: 2 for a
    # fully connected layer and 4 for a convolutional layer
    def __init__(self, num_features, num_dims):
        super().__init__()
        # if num_dims == 2:
        #     shape = (1, num_features)
        # else:
        #     shape = (1, num_features, 1, 1)
        shape = num_features
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))

        # The variables that are not model parameters are initialized to 0 and 1
        # self.running_mean = torch.zeros(shape)
        # self.running_var = torch.ones(shape)
        self.register_buffer('running_mean', torch.zeros(shape))
        self.register_buffer('running_var', torch.ones(shape))

    def forward(self, X):
        # If X is not on the main memory, copy moving_mean and moving_var to
        # the device where X is located
        if self.running_mean.device != X.device:
            self.running_mean = self.running_mean.to(X.device)
            self.running_var = self.running_var.to(X.device)
        # Save the updated running_mean and moving_var
        Y, self.running_mean, self.running_var = batch_norm(
            X, self.gamma, self.beta, self.running_mean,
            self.running_var, eps=1e-5, momentum=0.1)
        return Y


### Option 2

In [ ]:
class BatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(BatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # Learnable parameters
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        # Running mean and variance
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        if self.training:
            # Calculate mean and variance for the batch
            mean = x.mean([0, 2, 3], keepdim=True)
            var = x.var([0, 2, 3], keepdim=True, unbiased=False)
            # Update running mean and variance
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
        else:
            mean = self.running_mean
            var = self.running_var
        
        # Normalize
        x_normalized = (x - mean) / torch.sqrt(var + self.eps)
        # Scale and shift
        out = self.gamma.view(1, self.num_features, 1, 1) * x_normalized + self.beta.view(1, self.num_features, 1, 1)
        
        return out

### debug

In [ ]:
from batch_whitening import MyBatchNorm2d

In [ ]:
x = torch.randn(1, 10, 112, 112)
custom_bn = MyBatchNorm2d(num_features=10)
with torch.no_grad():
    out=custom_bn(x)

### Validation


In [ ]:
# Create a batch of 2D images (batch size, channels, height, width)
x = torch.randn(20, 10, 50, 50)
# x = torch.randn(1, 10, 112, 112)

# Our custom batch normalization layer
# custom_bn = BatchNorm(num_features=10, num_dims=4)
custom_bn = MyBatchNorm2d(num_features=10)

# PyTorch's built-in batch normalization layer
torch_bn = nn.BatchNorm2d(num_features=10)


# Copy the parameters from our custom layer to the built-in layer for a fair comparison
torch_bn.weight.data = custom_bn.gamma.data.clone()
torch_bn.bias.data = custom_bn.beta.data.clone()
torch_bn.running_mean = custom_bn.running_mean.clone()
torch_bn.running_var = custom_bn.running_var.clone()

# Forward pass
custom_bn_output = custom_bn(x)
torch_bn_output = torch_bn(x)

# Check if the outputs are close
assert torch.allclose(custom_bn_output, torch_bn_output, atol=1e-5), "The outputs are not close enough!"

print("Functional validation passed!")

In [ ]:
custom_bn = BatchNorm(num_features=10, num_dims=4)

In [ ]:
custom_bn.running_mean.shape

In [ ]:
custom_bn.running_var.shape

In [ ]:
x.var([0, 2, 3], keepdim=True, unbiased=False).shape

In [ ]:
custom_bn.training

In [ ]:
custom_bn.

# Batch Whitening

## implementation

In [ ]:
def cov_to_corr(cov_matrix):
    # Compute the standard deviations
    std = torch.sqrt(torch.diag(cov_matrix))
    
    # Compute the correlation matrix
    corr_matrix = cov_matrix / torch.outer(std, std)
    
    # Extract upper triangular part (excluding diagonal)
    upper_tri = torch.triu(corr_matrix, diagonal=1)
    
    # Compute average of cross-correlation coefficients
    avg_corr = upper_tri.abs().sum() / ((upper_tri.numel() - upper_tri.diag().numel())/2)
    
    return corr_matrix, avg_corr


def corr_to_cov(corr_matrix,std):
    # Compute the standard deviations
    D = torch.diag(std)
    # D=torch.outer(std,std)
    # cov=corr_matrix * torch.outer(std,std)
    return D@corr_matrix@D


def fix_corr(corr):
    a=0.9+0.1*torch.exp(-(abs(corr)/0.9)**10)
    a=a.clone()  # so not to lose the gradients in backprop
    torch.diagonal(a).fill_(1.0)
    return a*corr


### Choleski

#### diagonal Cholesky

In [ ]:
def batch_orthonorm_obsolete(X, gamma, beta, running_mean=None, running_cov=None, eps=1e-5, momentum=0.1):
    # Use is_grad_enabled to determine whether we are in training mode
    assert len(X.shape) in (2, 4)
    n_features = X.shape[1]

    if len(X.shape) == 2:
        # When using a fully connected layer, calculate the mean and
        # variance on the feature dimension
        shape = (1, n_features)
        mean = X.mean(dim=0)
        cov = torch.cov(X.T,correction=0)        
        # var = ((X - mean) ** 2).mean(dim=0)
    else:
        # When using a two-dimensional convolutional layer, calculate the
        # mean and covariance on the channel dimension (axis=1). Here we
        # need to maintain the shape of X, so that the broadcasting
        # operation can be carried out later
        shape = (1, n_features, 1, 1)
        mean = X.mean(dim=(0, 2, 3))
        Xtmp = X.view(X.shape[0],X.shape[1],-1)
        Xtmp = Xtmp.permute(1,0,2).reshape(X.shape[1],-1)
        cov = torch.cov(Xtmp,correction=0) 
        # var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
    # In training mode, the current mean and variance are used
    # Update the mean and variance using moving average
    running_mean = (1.0 - momentum) * running_mean + momentum * mean
    running_cov = (1.0 - momentum) * running_cov + momentum * cov
    L = torch.linalg.cholesky(running_cov + eps*torch.eye(n_features))
    if len(X.shape) == 2:
        X_hat = (X-running_mean.view(1,n_features)).T
        Y = torch.linalg.solve_triangular(L,X_hat,upper=False).T
    else:
        X_hat = X-running_mean.view(1,n_features,1,1)
        X_hat = X_hat.permute(1,0,2,3).reshape(X.shape[1],-1)
        Y = torch.linalg.solve_triangular(L,X_hat,upper=False).reshape(X.shape[1],X.shape[0],X.shape[2],X.shape[3]).permute(1,0,2,3)
    # Y = gamma.view(shape) * Y + beta.view(shape)  # Scale and shift
    return Y, running_mean.data, running_cov.data

def batch_orthonorm(X, gamma, beta, running_mean=None, running_cov=None, eps=1e-5, momentum=0.1):
    # Use is_grad_enabled to determine whether we are in training mode
    assert len(X.shape) in (2, 4)
    n_features = X.shape[1]

    if len(X.shape) == 2:
        # When using a fully connected layer, calculate the mean and
        # variance on the feature dimension
        shape = (1, n_features)
        mean = X.mean(dim=0)
        cov = torch.cov(X.T,correction=0)        
        # var = ((X - mean) ** 2).mean(dim=0)
    else:
        # When using a two-dimensional convolutional layer, calculate the
        # mean and covariance on the channel dimension (axis=1). Here we
        # need to maintain the shape of X, so that the broadcasting
        # operation can be carried out later
        shape = (1, n_features, 1, 1)
        mean = X.mean(dim=(0, 2, 3))
        Xtmp = X.view(X.shape[0],X.shape[1],-1)
        Xtmp = Xtmp.permute(1,0,2).reshape(X.shape[1],-1)
        cov = torch.cov(Xtmp,correction=0) 
        # var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
    # In training mode, the current mean and variance are used
    # Update the mean and variance using moving average
    if torch.is_grad_enabled():
        running_mean = (1.0 - momentum) * running_mean + momentum * mean
        running_cov = (1.0 - momentum) * running_cov + momentum * cov
        L = torch.linalg.cholesky(cov + eps*torch.eye(n_features))
        if len(X.shape) == 2:
            X_hat = (X-mean.view(1,n_features)).T
            Y = torch.linalg.solve_triangular(L,X_hat,upper=False).T
        else:
            X_hat = X-mean.view(1,n_features,1,1)
            X_hat = X_hat.permute(1,0,2,3).reshape(X.shape[1],-1)
            Y = torch.linalg.solve_triangular(L,X_hat,upper=False).reshape(X.shape[1],X.shape[0],X.shape[2],X.shape[3]).permute(1,0,2,3)
    else:
        L = torch.linalg.cholesky(running_cov + eps*torch.eye(n_features))
        if len(X.shape) == 2:
            X_hat = (X-running_mean.view(1,n_features)).T
            Y = torch.linalg.solve_triangular(L,X_hat,upper=False).T
        else:
            X_hat = X-running_mean.view(1,n_features,1,1)
            X_hat = X_hat.permute(1,0,2,3).reshape(X.shape[1],-1)
            Y = torch.linalg.solve_triangular(L,X_hat,upper=False).reshape(X.shape[1],X.shape[0],X.shape[2],X.shape[3]).permute(1,0,2,3)
    # Y = gamma.view(shape) * Y + beta.view(shape)  # Scale and shift
    return Y, running_mean.data, running_cov.data

class BatchWhitening(nn.Module):
    # num_features: the number of outputs for a fully connected layer or the
    # number of output channels for a convolutional layer. num_dims: 2 for a
    # fully connected layer and 4 for a convolutional layer
    def __init__(self, num_features,momentum=0.1):
        super().__init__()
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.momentum = momentum
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        # The variables that are not model parameters are initialized to 0 and 1
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_cov', torch.eye(num_features))

    def forward(self, X):
        # If X is not on the main memory, copy moving_mean and moving_var to
        # the device where X is located
        if self.running_mean.device != X.device:
            self.running_mean = self.running_mean.to(X.device)
            self.running_cov = self.running_cov.to(X.device)
        # Save the updated running_mean and moving_var
        # Y, self.running_mean, self.running_var = batch_orthonorm(
        #     X, self.gamma, self.beta, self.running_mean,
        #     self.running_cov, eps=1e-5, momentum=0.1)
        Y, self.running_mean, self.running_cov = batch_orthonorm(
            X, self.gamma, self.beta, self.running_mean,
            self.running_cov, eps=1e-5, momentum=self.momentum)

        return Y


#### Block-diagonal Cholesky

In [ ]:
def get_grp_ch(num_features,num_groups=1,num_channels=-1):
    if num_channels == -1:
        num_channels = (num_features - 1) // num_groups + 1
    num_groups = num_features // num_channels
    while num_features % num_channels != 0:
        # num_channels //= 2
        num_channels -= 1
        num_groups = num_features // num_channels
    assert num_groups > 0 and num_features % num_groups == 0, "num features={}, num groups={}".format(num_features,
        num_groups)
    return num_groups,num_channels

def block_cholesky_batch(X, running_mean=None, running_cov=None, n_channels=-1, eps=1e-5, momentum=0.1,cov_warmup=False):
    # Use is_grad_enabled to determine whether we are in training mode
    assert len(X.shape) in (2, 4)
    n_features = X.shape[1]
    if n_channels==-1:
        n_channels=n_features
    n_groups=n_features//n_channels
    x = X.transpose(0, 1).contiguous().view(n_groups, n_channels, -1)    
    cov_I = torch.eye(n_channels).expand(n_groups, n_channels, n_channels).to(running_cov.device)     
    _, d, m = x.size()
    mean = x.mean(-1, keepdim=True)
    xc = x - mean
    cov = torch.baddbmm(cov_I, xc, xc.transpose(1, 2), beta=eps, alpha=1. / m)

    # In training mode, the current mean and variance are used
    # Update the mean and variance using moving average
    # running_mean = (1.0 - momentum) * running_mean + momentum * mean
    running_mean = mean         # no running mean (alpha = momentum = 1)
    
    # during warmup, we're not updating running_cov but using a statistics of current batch 
    if cov_warmup:
        # x_var = torch.diag_embed(torch.diag(cov))
        x_var = torch.eye(d).unsqueeze(0)*cov
        running_cov = (1.0 - momentum) * x_var + momentum * cov
    # when warm up is done, running_cov is updated only during training
    elif torch.is_grad_enabled():    
        running_cov = (1.0 - momentum) * running_cov + momentum * cov
    # note that we're using running_cov also during training
    # L = torch.linalg.cholesky(running_cov + eps*cov_I)
    L = torch.linalg.cholesky(running_cov)
    if len(X.shape) == 2:
        Y = torch.linalg.solve_triangular(L,xc,upper=False).reshape(X.shape[1],X.shape[0]).permute(1,0)
    else:
        Y = torch.linalg.solve_triangular(L,xc,upper=False).reshape(X.shape[1],X.shape[0],X.shape[2],X.shape[3]).permute(1,0,2,3)
    return Y, running_mean.data, running_cov.data


class BatchWhitening(nn.Module):
    # num_features: the number of outputs for a fully connected layer or the
    # number of output channels for a convolutional layer. num_dims: 2 for a
    # fully connected layer and 4 for a convolutional layer
    def __init__(self, num_features,num_groups=1, num_channels=-1, momentum=0.1):
        super().__init__()
        self.num_groups,self.num_channels=get_grp_ch(num_features,num_groups,num_channels)
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.momentum = momentum
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        # The variables that are not model parameters are initialized to 0 and 1
        self.register_buffer('running_mean', torch.zeros(self.num_groups, self.num_channels, 1))
        self.register_buffer('running_cov', torch.eye(self.num_channels).expand(self.num_groups, self.num_channels, self.num_channels))

    def forward(self, X):
        # If X is not on the main memory, copy moving_mean and moving_var to
        # the device where X is located
        if self.running_mean.device != X.device:
            self.running_mean = self.running_mean.to(X.device)
            self.running_cov = self.running_cov.to(X.device)
        # Save the updated running_mean and moving_var
        Y, self.running_mean, self.running_cov = block_cholesky_batch(
            X, self.running_mean, self.running_cov, self.num_channels, eps=1e-5, momentum=self.momentum)

        return Y


In [ ]:
num_groups=4
num_channels=3
xx=torch.eye(num_channels).expand(num_groups, num_channels, num_channels)
xx.shape

In [ ]:
num_features=32
num_groups=6
num_channels=-1
get_grp_ch(num_features,num_groups=num_groups,num_channels=num_channels)

### IterNorm

The algorithm :  (taken from [the paper](https://arxiv.org/abs/1904.03441) )  
![iternorm](./iternorm_algo.png)

In [ ]:
from torch.nn import Parameter

def fix_corr(corr):
    a=0.9+0.1*torch.exp(-(abs(corr)/0.9)**10)
    a=a.clone()  # so not to lose the gradients in backprop
    torch.diagonal(a).fill_(1.0)
    return a*corr

def get_grp_ch(num_features,num_groups=1,num_channels=-1):
    if num_channels == -1:
        num_channels = (num_features - 1) // num_groups + 1
    num_groups = num_features // num_channels
    while num_features % num_channels != 0:
        num_channels //= 2
        num_groups = num_features // num_channels
    assert num_groups > 0 and num_features % num_groups == 0, "num features={}, num groups={}".format(num_features,
        num_groups)
    return num_groups,num_channels


class IterNorm(nn.Module):
    def __init__(self, num_features, num_groups=1, num_channels=-1, T=5, dim=4, eps=1e-5, momentum=0.1, affine=True, *args, **kwargs):
        super(IterNorm, self).__init__()
        # assert dim == 4, 'IterNorm is not support 2D'
        self.T = T
        self.eps = eps
        self.momentum = momentum
        self.num_features = num_features
        self.affine = affine
        self.dim = dim
        self.fix_cov=kwargs.get('fix_cov',False)

        num_groups,num_channels=get_grp_ch(num_features,num_groups,num_channels)
        self.num_groups = num_groups
        self.num_channels = num_channels
        shape = [1] * dim
        shape[1] = self.num_features
        if self.affine:
            self.weight = Parameter(torch.Tensor(*shape))
            self.bias = Parameter(torch.Tensor(*shape))

        if not self.affine:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)

        self.register_buffer('running_mean', torch.zeros(num_groups, num_channels, 1))
        # running whiten matrix
        self.register_buffer('running_wm', torch.eye(num_channels).expand(num_groups, num_channels, num_channels))
        self.reset_parameters()

    def reset_parameters(self):
        # self.reset_running_stats()
        if self.affine:
            self.weight.data.fill_(1.0)
            self.bias.data.fill_(0.0)

    def forward(self, X: torch.Tensor):
        eps = 1e-5
        momentum = self.momentum

        nc = self.num_channels
        T = self.T
        g = X.size(1) // nc
        x = X.transpose(0, 1).contiguous().view(g, nc, -1)
        _, d, m = x.size()
        if self.training:
            # calculate centered activation by subtracted mini-batch mean
            mean = x.mean(-1, keepdim=True)
            xc = x - mean
            # calculate covariance matrix
            P = [None] * (T + 1)
            P[0] = torch.eye(d).to(X).expand(g, d, d)
            # Sigma = torch.baddbmm(eps, P[0], 1. / m, xc, xc.transpose(1, 2))
            Sigma = torch.baddbmm(P[0], xc, xc.transpose(1, 2), beta=eps, alpha=1. / m)  # =torch.cov(xc,correction=0)
            if self.fix_cov:
                Sigma[0]=fix_corr(Sigma[0])
            # reciprocal of trace of Sigma: shape [g, 1, 1]
            rTr = (Sigma * P[0]).sum(1, keepdim=True).sum(2, keepdim=True).reciprocal_()
            Sigma_N = Sigma * rTr
            for k in range(T):
                # P[k + 1] = torch.baddbmm(1.5, P[k], -0.5, P[k].bmm(P[k]).bmm(P[k]), Sigma_N)
                P[k + 1] = torch.baddbmm(P[k], P[k].bmm(P[k]).bmm(P[k]), Sigma_N, beta=1.5, alpha=-0.5)
            wm = P[T].mul_(rTr.sqrt())  # whiten matrix: the matrix inverse of Sigma, i.e., Sigma^{-1/2}
            # self.running_mean += momentum * ( mean.detach() - self.running_mean)
            # self.running_wm += momentum * ( wm.detach() - self.running_wm)
            self.running_mean = (1-momentum)*self.running_mean + momentum * mean.detach()
            self.running_wm = (1-momentum)*self.running_wm + momentum * wm.detach() 
        else:
            xc = x - self.running_mean
            wm = self.running_wm
        xn = wm.matmul(xc)
        X_hat = xn.view(X.size(1), X.size(0), *X.size()[2:]).transpose(0, 1).contiguous()
        
        # affine
        if self.affine:
            X_hat = X_hat * self.weight
            X_hat = X_hat + self.bias
        return X_hat

    def extra_repr(self):
        return '{num_features}, num_channels={num_channels}, T={T}, eps={eps}, dim={dim}, ' \
               'momentum={momentum}, affine={affine}'.format(**self.__dict__)

from batch_whitening import IterNormMod,iter_norm_batch



In [ ]:
n_grp,n_ch=get_grp_ch(65,3)
print(f'{n_grp} groups, {n_ch} channels')

## Validation

from [wikipedia](https://en.wikipedia.org/wiki/Covariance_matrix):  
The covariance matrix is given by:  
![cov](cov.png)

and the correlation matrix is given by:  
![corr](corr_mat.png)

so if we have a matrix of `N` samples of a random vector $x \in \mathbb{R}^C$ , we can compute the covariance of the random vector as follows:
`corr_matrix = torch.corrcoef(x_f)`  
and we can also compute the covariance matrix with: `cov_matrix = x.cov()`   

we can also get `corr_matrix` from `cov_matrix` using the following function:

In [ ]:
import numpy as np

def generate_well_conditioned_covariance(n, condition_number=2):
    # Step 1: Generate a random matrix
    # L = np.random.rand(n, n)
    L = torch.rand(n, n)
    
    # Step 2: Make it lower triangular with positive diagonal
    L = torch.tril(L) + n * torch.eye(n)
    
    # Step 3: Construct the symmetric positive definite matrix
    A = L@L.T
    
    # Step 4: Adjust the condition number
    # Scale the matrix to have a desired condition number
    u, s, vh = torch.linalg.svd(A)
    s = torch.linspace(s[0], s[0] / condition_number, len(s))
    A = (u * s)@vh
    
    return A


In [ ]:
cov=generate_well_conditioned_covariance(3,10)
print(cov)
print(f"condition number: {torch.linalg.cond(cov)}")
torch.linalg.eig(cov)[0]

In [ ]:
# the following compute the rank of the matrix and the average of the correlation coefficients in the correlation matrix:
# x is expected to have the following shape: [B,C,H,W] if it has 4 dimensions, [B,C] if it has 2 dimensions 
def rank_and_avg_corr(x):
    if len(x.shape)==4:  # conv
        # flatten x from [B,C,H,W] to [C,B*H*W]
        x_f= x.permute(1,0,2,3).reshape(x.shape[1],-1)
    else:
        # change from [B,C] to [C,B]
        x_f=x.T
    # compute corr matrix
    corr_matrix = torch.corrcoef(x_f)
    # Extract upper triangular part (excluding diagonal)
    upper_tri = torch.triu(corr_matrix, diagonal=1)
    # Compute average of cross-correlation coefficients
    avg_corr = upper_tri.sum() / (upper_tri.numel() - upper_tri.diag().numel())
    rank = torch.linalg.matrix_rank(x_f)/x.shape[1]
    return rank,avg_corr


#### 4D tensor

In [ ]:
# Create a batch of 2D images (batch size, channels, height, width)
num_features = 3
n_samples=100
m = torch.randint(10,100,(1,num_features)).float()
c = torch.randint(1,10,(1,num_features))
# cov = c.T@c + 0.1*torch.eye(num_features)       # ill conditioned 
cov = generate_well_conditioned_covariance(num_features,10000)  # well conditioned

print(f'generating a tensor of shape ({n_samples}, {num_features},32,32) with mean {m} and covariance \n {cov}:')
x = torch.randn(n_samples, num_features, 32, 32).permute(1,0,2,3).reshape(num_features,-1).T
L = torch.linalg.cholesky(cov.float())
xc= x@L.T + m
# xc[:,-1]=0.5*xc[:,-2]+0.5*xc[:,-3]
# print(f'actual mean and cov: {xc.mean(0)},\n {torch.cov(xc.T,correction=0)}')
print(f'actual mean and cov: {xc.mean(0)},\n {torch.cov(xc.T)}')
x_c = xc.permute(1,0).reshape(num_features,n_samples,32,32).permute(1,0,2,3)
# flatten x_c to [num_featurs,num_samples]:
x_c_f= x_c.permute(1,0,2,3).reshape(x_c.shape[1],-1)



In [ ]:
# compute the cov from x_c:
x_c_cov = x_c_f.cov()
print(x_c_cov)
print(f' the eigen values of the cov matrix are : {torch.linalg.eigh(x_c_cov)[0]}')
print(f"condition number: {torch.linalg.cond(x_c_cov)}")
print("\n \n correlation matrix and average cross correlation:") 
cov_to_corr(x_c_cov)

In [ ]:
# compute the correlation coefficient matrix
print(torch.corrcoef(x_c_f))
# test the conversion function: should be the same output
x_c_corr=cov_to_corr(x_c_cov)[0]
x_c_corr

In [ ]:
corr_to_cov(x_c_corr,x_c_f.std(axis=-1))

In [ ]:
print(xc.shape)
print(x_c.shape)
print(x_c_f.shape)

##### Validating Batch Whitening layer

In [ ]:
# Our custom batch normalization layer
bw_layer = BatchWhitening(num_features,momentum=1)

# Forward pass
x_w = bw_layer(x_c)   # expecting x_c.shape=[N,C,H,W] 

x_w_cov = x_w.permute(1,0,2,3).reshape(x.shape[1],-1).cov()
print(f"x_w_cov matrix: \n {x_w_cov}")
print(f"eigen values of x_w_cov matrix: \n {torch.linalg.eigh(x_w_cov)[0]}")
print(f"condition number: {torch.linalg.cond(x_w_cov)}")
x_w_corr,x_w_avgxcorr = cov_to_corr(x_w_cov)
print(f"correlation matrix and average cross correlation: \n {x_w_corr} \n {x_w_avgxcorr}")
# Check if the outputs are indeed orthonormal
assert torch.allclose(x_w_cov, torch.eye(num_features), atol=1e-2), "The outputs are not close enough!"
print("Functional validation passed!")

In [ ]:
# since the layer also normalized the input, the correlation coefficient matrix should be nearly the same (up to approx errors) as the covariance matrix. lets check it:
print(cov_to_corr(x_w_cov))
# alternatice way to compute: - should produce the same output
print(torch.corrcoef(x_w.permute(1,0,2,3).reshape(x.shape[1],-1)))

In [ ]:
x_c_cov = x_c.permute(1,0,2,3).reshape(x.shape[1],-1).cov()
# we can also compute x_c_cov as follows
# torch.cov(x_c.permute(1,0,2,3).reshape(x.shape[1],-1))
print(x_c_cov)
cov_to_corr(x_c_cov)

In [ ]:
# compute the rank
torch.linalg.matrix_rank(x_c.permute(1,0,2,3).reshape(x.shape[1],-1))/x_c.shape[1]

##### Validating iter norm 

In [ ]:
ItN = IterNormMod(3, num_groups=1, T=11, momentum=1, affine=False,fix_cov=True)
print(ItN)
ItN.train()
x_c.requires_grad_()
x_w = ItN(x_c)
print(x_w.shape)
x_w_cov = x_w.permute(1,0,2,3).reshape(x_w.shape[1],-1).cov()

print(f"x_w_cov matrix: \n {x_w_cov}")
print(f"eigen values of x_w_cov matrix: \n {torch.linalg.eigh(x_w_cov)[0]}")
print(f"condition number: {torch.linalg.cond(x_w_cov)}")
x_w_corr,x_w_avgxcorr = cov_to_corr(x_w_cov)
print(f"correlation matrix and average cross correlation: \n {x_w_corr} \n {x_w_avgxcorr}")

In [ ]:
torch.linalg.cond(x_w_cov)

In [ ]:
len(x_w_cov.diag())

In [ ]:
a=torch.ones_like(x_w_cov)*0.9
a

In [ ]:
a.fill_diagonal_(1.0)

In [ ]:
a

In [ ]:
# torch.corrcoef(x_w.permute(1,0,2,3).reshape(x_w.shape[1],-1))
cov_to_corr(x_w_cov)

#### 2D tensor

In [ ]:
# Create a batch of vectors (B, num_features)
# num_features = 4
num_features = 64
m = torch.randint(10,100,(1,num_features)).float()
c = torch.randint(1,10,(1,num_features))
# cov = c.T@c + 0.1*torch.eye(num_features)
cov = generate_well_conditioned_covariance(num_features,100)


print(f'generating a tensor of shape (B, {num_features}) with mean {m} and covariance \n {cov}:')
x = torch.randn(1000, num_features)
# x = torch.randn(128, num_features)
L = torch.linalg.cholesky(cov.float())
xc= x@L.T + m 
print(f'tensor xc.shape={xc.shape}')
print(f'actual mean and cov: {xc.mean(0)},\n {torch.cov(xc.T)}')

# print(f'rank and average correlation of x: {rank_and_avg_corr(x)}')
# print(f'rank and average correlation of xc: {rank_and_avg_corr(xc)}')


In [ ]:
x_c_cov=torch.cov(xc.T)
print(f"condition number: {torch.linalg.cond(x_c_cov)}")
print("\n \n correlation matrix and average cross correlation:") 
cov_to_corr(x_c_cov)

In [ ]:
print(f'eigen values of x.cov: {torch.linalg.eig(torch.cov(x.T))[0]}')
print(f'eigen values of xc.cov: {torch.linalg.eig(torch.cov(xc.T))[0]}')


In [ ]:
C=torch.corrcoef(xc.T).detach().numpy()

##### Batch whitening (Cholesky)

In [ ]:
# Our custom batch normalization layer
bw_layer = BatchWhitening(num_features,num_groups=8,momentum=1)

print('cov matrix of xc.T')
print(torch.cov(xc.T))

# Forward pass - expecting xc.shape=[N,D]
x_w = bw_layer(xc)

print('cov matrix of x_w.T')
print(torch.cov(x_w.T))

# Check if the outputs are indeed orthonormal
assert torch.allclose(x_w.T.cov(), torch.eye(num_features), atol=1e-2), "The outputs are not close enough!"

print("Functional validation passed!")

In [ ]:
x_w_cov=torch.cov(x_w.T)
x_w_cov.shape

In [ ]:
k=2
# x_w_cov[8*k:8*(k+1),8*k:8*(k+1)]
[torch.allclose(x_w_cov[8*k:8*(k+1),8*k:8*(k+1)], torch.eye(8), atol=1e-2) for k in range(8)]

In [ ]:
# print the correlation matrix of the whitened signal and the average cross correlation
cov_to_corr(x_w.T.cov())

In [ ]:
# should be the same correlation matrix as previous cell
torch.corrcoef(x_w.T)


##### Iternorm

using the same signal as above

In [ ]:
# using the same signal 
print(f' xc.shape={xc.shape}')

In [ ]:
ItN = IterNorm(num_features, num_groups=1, T=10,  momentum=1, affine=False)
print(ItN)
ItN.train()
xc.requires_grad_()
y = ItN(xc)  # y.shape=xc.shape=[N,D]
# the following is an alternative computation of y's cov matrix (implements what cov is doing behind the scene with correction=0)
z = y.transpose(0, 1).contiguous().view(x.size(1), -1) # z.shape=[D,N]
y_cov = z.matmul(z.t()) / z.size(1)
print("y_cov:")
print(y_cov)  # the outcome is the cov matrix at shape [D,D]

cov_to_corr(y_cov)

In [ ]:
# alternative computation 
x_w = ItN(xc)
print(x_w.shape)
x_w_cov = x_w.T.cov(correction=0)
print(x_w_cov)

# torch.corrcoef(x_w.T)
cov_to_corr(x_w_cov)

In [ ]:
xc.T.cov()

In [ ]:
xc_corr=cov_to_corr(xc.T.cov())[0]
xc_corr

In [ ]:
c2c=corr_to_cov(xc_corr,xc.std(axis=0))
c2c

In [ ]:
# torch.std(xc,dim=0)
xc.std(axis=0)

### Debug

In [ ]:
M = torch.randn(10, 3, 5)
A = torch.zeros_like(M)
batch1 = torch.randn(10, 3, 4)
batch2 = torch.randn(10, 4, 5)
B=torch.baddbmm(M, batch1, batch2,out=A)

In [ ]:
def get_a(c):
    a=0.9+0.1*torch.exp(-(c/0.9)**10)
    torch.diagonal(a).fill_(1.0)
    return a


In [ ]:
# C=torch.corrcoef(xc.T).detach().numpy()
C=torch.corrcoef(xc.T)
print(C)

In [ ]:
import numpy as np

In [ ]:
def get_batch_whitening_config_h(N, H, W, C, momentum=0.99, threshold=0.01):
    """Set Batch Whitening configuration based on number of samples
    
    Args:
        N (int): Number of samples
        H (int): Height of the image
        W (int): Width of the image
        C (int): Number of channels
        threshold (float): Threshold for using BatchWhitening

    The mechanism:
    the effective number of samples is N*H*W/(1-momentum)
    this number should be >= blk_size*blk_size/2*threshold^2

    we can control blk_size. so we want to find what is the blk_size that satisfies the condition above.
    so we solve for blk_size in the equation:
    N*H*W/(1-momentum) >= blk_size*blk_size/2*threshold^2
    blk_size >= sqrt(2*N*H*W/(1-momentum)*threshold^2)
    now, if blk_size < 2 , we set blk_size to 1, which means using batchnorm.
    if blk_size > C we clip it to C which means using the whole channel as a group.
    in between, we use blk_size as nearest power of 2 to the blk_size.
    """
    print('-----------BW triage------------ \n')
    n_samples = N * H * W / (1-momentum)

    blk_size = int(np.sqrt(2*n_samples*threshold*threshold))
    print(f'raw block size: {blk_size}')
    new_mom=momentum
    if blk_size < 2:
        blk_size = 1
    elif blk_size > C:
        blk_size = C
        new_mom=1-(2*threshold*threshold*N*H*W)/(C*C)
    else:
        blk_size = 2**int(np.log2(blk_size))
    print(f'N,H,W,C,mu={N,H,W,C,momentum} --> blk_size = {blk_size}, momentum={new_mom}')
    return blk_size,new_mom


In [ ]:
from batch_whitening import get_batch_whitening_config2

In [ ]:
blk_size,mu=get_batch_whitening_config2(32,7,7,1152,0.99)
print(blk_size)
print(mu)